## Weather CNN

In [173]:
import math
import pickle
import os
import numpy as np
import pandas as pd

from datetime import datetime as dt

In [174]:
def create_weather_dict(directory, scaled_weather):
    '''
    Create a dictionary of weather data from a pickled file
    Args:
        - directory: path to weather pickle file
        - scaled_weather: True/False to scale using max value
    Returns:
        - weather_data: dictionary of key (day of year) and value (dictionary of key (weather parameter) 
        and value (matrix of value for each pixel))
    '''

    path = os.path.abspath(directory)
    
    weather_file = ''
    
    for f in os.listdir(path):
        if f.endswith('.pickle'):
            weather_file = path + '/' + f
    
    weather = pd.read_pickle(weather_file)
    
    weather_dict = {}
    
    for k, v in weather.items():
        weather_dict[k] = {}
        
        for att, matrix in v.items():
            mat = np.nan_to_num(matrix)
            weather_dict[k][att] = mat
     
    weather_data = {}

    for k, v in weather_dict.items():
        doy = dt.strptime(k, "%Y-%m-%d").strftime("%j")
        weather_data[doy] = v
    
    # scale weather data
    vals = list(weather_data.values())[0]
    weather_atts = list(vals.keys())
    max_values = dict.fromkeys(weather_atts, 0)
    
    if scaled_weather == True:
        
        for k, v in weather_dict.items():

            for weather_att, matrix in v.items():
                max_val = matrix.max()
                if max_val > max_values[weather_att]:
                    max_values[weather_att] = max_val
    
    return weather_data, max_values

In [175]:
def fetch_weather_data(max_values, scaled_weather, day_of_year, x, y):
    '''
    Fetch weather data for the relevant day and pixel.
    
    Args:
        - day_of_year: day of the year (1-365)
        - x: x-coordinate of matrix
        - y: y-coordinate of matrix
    Returns:
        - weather_data: an array of relevant weather data for that pixel
    '''
    
    weather_list = []
    
    day_weather = weather_data.get(day_of_year)
    
    if day_weather is None:
        return None
    else:
        for k, v in day_weather.items():
            if scaled_weather == True:
                max_val = max_values.get(k, 1)
                val = v[x,y]/max_val
                
                if math.isnan(val):
                    weather_list.append(0)
                else:
                    weather_list.append(val)
            else:
                weather_list.append(v[x,y])
    
    return weather_list
    

In [177]:
weather_directory = 'weather_data'
scaled_weather = False
weather_data, max_values = create_weather_dict(weather_directory, scaled_weather)
fetch_weather_data(max_values, scaled_weather, '100', 0, 1)

[0.48816009012838274,
 0.0,
 17.19940659358134,
 12.940884680607965,
 0.8710503729338492,
 3.8544342011009194,
 260.15575366420785,
 0.8996613262128507]

In [184]:
count  = 0
b = 0

for k, v in weather_data.items():
    for att, mat in v.items():
        if mat.shape == (470,456):
            count += 1
        else:
            b += 1

In [185]:
print(count)
print(b)

768
0
